# Python爬虫教程

课程来源：[Python超强爬虫8天速成（完整版）爬取各种网站数据实战案例](https://www.bilibili.com/video/BV1ha4y1H7sx?p=6&spm_id_from=pageDriver)

## 使用场景

通用爬虫：抓取系统的组成部分，抓取一张页面数据

聚焦爬虫：在通用的基础上，抓取特定内容

增量式爬虫：检测网站中数据更新

## http协议
服务器与客户端进行数据交互的一种形式

常用请求头信息

- User-Agent：请求载体的身份标识

- Connection：请求完毕后，是断开连接还是保持

常用响应头协议
- Content-Type：服务器响应客户端的数据类型

## https协议

- 安全的超文本协议

加密方式

- 对称密钥加密

- 非对称密钥加密

- 证书密钥加密

---

## requests模块

作用：模拟浏览器发送请求

如何使用：

- 指定url

- 发起请求

- 获取响应数据

- 持久化存储

In [ ]:
#需求：爬取搜狗首页
import requests
#指定url
url = 'https://www.sogou.com/'
#发起请求
#返回响应对象
response = requests.get(url = url)
#获取响应数据
#获取字符串形式的响应数据
page_text = response.text
print(page_text)
#持久化存储
with open('./sougou.html', 'w', encoding ='utf-8') as fp:
    fp.write(page_text)
    print('爬取结束！')

## 案例

- 指定搜索结果页面

- 破解百度翻译

- 豆瓣电影分类排行榜

- 肯德基餐厅查询

- 国家药监局相关数据

## 指定搜索结果页面

### User-Agent伪装

UA检测：

门户网站的服务器检测对应的请求对应载体身份标识，
若检测到载体的身份标识是某一浏览器，则为正常请求，
若检测载体的身份标识为非浏览器，则异常，可能拒绝请求。

查看方式：

`Shift + Ctrl + I`打开浏览器开发者工具，点击网络，选中网站，点击标头，找到User-Agent

In [ ]:
#指定搜索结果页面
import requests
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
url = 'https://www.sogou.com/web'
#处理url携带的参数：封装到字典中
kw = input('enter a word:')
param = {
    'query' : kw
}
#对指定url发起请求是携带参数的，且请求过程中处理了参数
response = requests.get(url = url, params = param, headers = headers)
page_text = response.text
filename = kw + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

## 破解百度翻译

刷新页面的一部分

### 相关知识

- post请求（携带了参数）

- 响应数据是一组

### 操作

1. `Shift + Ctrl + I`打开开发者工具，点击网络，点击Fetch/XHR

2. 点击名称，点击负载，找到显示关键词的组

3. 返回标头，复制url

4. 在标头中找到Content-Type，显示json则为json方法

In [ ]:
#破解百度翻译
import requests
import json
post_url = 'https://fanyi.baidu.com/sug'
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
#post请求参数处理（与get相同）
word = input('enter a word:')
data = {
    'kw' : word
}
#请求发送
response = requests.post(url = post_url, data = data, headers = headers)
#获取相应数据:json方法返回的是object(响应数据为json类型，才能使用)
dic_obj = response.json()
#持久化存储
filename = word + '.json'
fp = open(filename, 'w', encoding = 'utf-8')
json.dump(dic_obj, fp = fp, ensure_ascii = False)
print('over!')

## 豆瓣电影分类排行榜

下拉页面会进行部分刷新，如增加电影数量没滚动条位置变化

In [ ]:
#豆瓣电影分类排行榜
import requests
import json
url = 'https://movie.douban.com/j/chart/top_list'
param = {
    'type' : '24',
    'interval_id' : '100:90',
    'action' : '',
    'start' : '0', #从库中第几部电影开始取
    'limit' : '20', #单次请求取出的个数
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.get(url = url, params = param, headers = headers)
list_data = response.json()
fp = open('./douban.json', 'w', encoding = 'utf-8')
json.dump(list_data, fp = fp, ensure_ascii = False)
print('over!')

## 作业：肯德基餐厅查询

重点：

1. 确定网址的url

2. 确定post请求还是get请求

3. 确定响应数据是text还是json

In [ ]:
#作业：肯德基餐厅查询
import requests
url = 'https://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'
word = input('enter a city:')
data = {
    'cname' : '',
    'pid' : '',
    'keyword' : word,
    'pageIndex' : '1',
    'pageSize' : '10',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.post(url = url, data = data, headers = headers)
page_text = response.text
filename = word + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

## 国家药监局基于化妆品生产许可证相关数据

网址：[化妆品生产许可信息管理平台](http://scxk.nmpa.gov.cn:81/xk/)

需求：需要爬取企业的具体信息，非主业列表

并不一定能直接从地址栏中请求得到

可能是**动态加载**出来的数据

对应企业的数据是通过ajax动态请求得到的

**<font color=red>未解决</font>**

In [ ]:
#国家药监局基于化妆品生产许可证相关数据
import requests
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'
data = {
    'on': 'true',
    'page': '1',
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
id_list = []
json_ids = requests.post(url = url, headers = headers, data = data)
for dic in json_ids['list']:
    id_list.append(dic['ID'])

---

## 数据解析

聚焦爬虫：爬取页面指定内容

分类：正则、bs4、**xpath**

原理：

- 解析局部文本内容在标签之间或标签对应的属性中存储

- 指定标签进行定位

- 标签或标签属性中存储的数据值进行提取（解析）

流程：

1. 指定url

2. 发起请求

3. 获取相应数据

4. 数据解析

5. 持久化存储

In [ ]:
#爬取图片数据
import requests
url = 'http://mari.hzau.edu.cn/__local/B/C9/4C/40EE6E9C872EBEF7B0E864D8540_34AE630B_15EF3.jpg'
#content返回二进制图像数据
#text字符串，content二进制，json对象
img_data = requests.get(url = url).content
with open('./image.jpg', 'wb') as fp:
    fp.write(img_data)

In [ ]:
#爬取所有图片数据
import requests
import re
import os
#创建文件夹
if not os.path.exists('./marilibs'):
    os.mkdir('./marilibs')
#爬取页面信息
url = 'http://mari.hzau.edu.cn/info/1003/3711.htm'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
page_text = requests.post(url = url, headers = headers).text
#获取图像数据
#正则表达式
ex = '<p style="text-align: center"><img src="(.*?)" width'
img_src_list = re.findall(ex, page_text, re.S)
print(img_src_list)
#拼接图片地址
for src in img_src_list:
    src = 'http://mari.hzau.edu.cn' + src
    img_data = requests.get(url = src, headers = headers).content
#存储图片
    img_name = src.split('/')[-1]
    imgPath = './marilibs/' + img_name
    with open(imgPath, 'wb') as fp:
        fp.write(img_data)
        print(img_name, '下载成功！')